In [ ]:
import torch
import torch.nn as nn

The link to the paper: `https://static.googleusercontent.com/media/research.google.com/en//pubs/archive/43022.pdf`

The salient features:

1. Uses 1x1 cnn for dimensionality reduction

Inception net

![](https://i.ytimg.com/vi/KfV8CJh7hE0/maxresdefault.jpg)

So it has multiple conv block and inception block lets build them one by one
# Conv block & Inception block

In [ ]:
class conv_block(nn.Module):
    def __init__(self, in_channels, out_channels, **kwargs):
        super(conv_block, self).__init__()
        self.relu = nn.ReLU()
        self.cnn = nn.Conv2d(in_channels, out_channels, **kwargs)
        self.batchnorm = nn.BatchNorm2d(out_channels)
    
    def forward(self, x):
        return self.relu(self.batchnorm(self.cnn(x)))

In [ ]:
class inception_block(nn.Module):
    def __init__(self, in_channels, out1x1, red3x3, out3x3, red5x5, out5x5, out1x1pool):
        super(inception_block, self).__init__()
        
        self.branch1 = conv_block(in_channels, out1x1, kernel_size=1)
        
        self.branch2 = nn.Sequential(
            
            conv_block(in_channels, red3x3, kernel_size=1),
            conv_block(red3x3, out3x3, kernel_size=3, padding=1)
        )
        
        self.branch3 = nn.Sequential(
            conv_block(in_channels, red5x5, kernel_size=1),
            conv_block(red5x5, out5x5, kernel_size=5, padding=2)
        )
        
        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            conv_block(in_channels, out1x1pool, kernel_size=1)
        )
        
    def forward(self, x):
        return torch.cat([self.branch1(x), self.branch2(x), self.branch3(x), self.branch4(x)], 1)

![](https://media.geeksforgeeks.org/wp-content/uploads/20200429201421/Inception-layer-by-layer.PNG)

In [ ]:
class GoogLeNet(nn.Module):
    def __init__(self, in_channels=3, num_classes=104):
        super(GoogLeNet,self).__init__()
        self.conv1 = conv_block(in_channels, 64,kernel_size=(7,7), stride=(2,2),padding=(3,3))
        self.maxpool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.conv2 = conv_block(64, 192, kernel_size=3, stride=1, padding=1)
        self.maxpool2 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.inception3a = inception_block(192, 64, 96, 128, 16, 32, 32)
        self.inception3b = inception_block(256, 128, 128, 192, 32, 96, 64)
        self.maxpool3 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.inception4a = inception_block(480, 192, 86, 208, 16, 48, 64)
        self.inception4b = inception_block(512, 160, 112, 224, 24, 64, 64)
        self.inception4c = inception_block(512, 128, 128, 256, 24, 64, 64)
        self.inception4d = inception_block(512, 112, 144, 288, 32, 64, 64)
        self.inception4e = inception_block(528, 256, 160, 320, 32, 128, 128)
        self.maxpool4 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.inception5a = inception_block(832, 256, 160, 320, 32, 128, 128)
        self.inception5b = inception_block(832, 384, 192, 384, 48, 128, 128)
        
        self.avgpool = nn.AvgPool2d(kernel_size=7, stride=1)
        self.dropout = nn.Dropout(p=0.4)
        
        self.fc1 = nn.Linear(1024,num_classes)
    
    def forward(self,x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        
        x = self.conv2(x)
        x = self.maxpool2(x)
        
        x = self.inception3a(x)
        x = self.inception3b(x)
        x = self.maxpool3(x)
        
        x = self.inception4a(x)
        x = self.inception4c(x)
        x = self.inception4d(x)
        x = self.inception4e(x)
        x = self.maxpool4(x)
        
        x = self.inception5a(x)
        x = self.inception5b(x)
        x = self.avgpool(x)
        x = self.dropout(x)
        
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)
        
        return x
        
        
        
        
    

In [ ]:
model = GoogLeNet()

In [ ]:
x = torch.randn(10, 3, 224,224)
y = model(x)
y.shape

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

img = Image.open('../input/104-flowers-garden-of-eden/jpeg-224x224/train/balloon flower/10108.jpeg')
img = np.array(img)
img = torch.tensor(img)
plt.imshow(img)
img.shape


In [ ]:
x = img.permute(2,0,1)
x = x.unsqueeze(0)
x = x.float()
print(x.shape)
y = model(x)
"output shape: " , y.shape

lets use flower dataset

In [ ]:
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms


In [ ]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
train_transform = transforms.Compose([transforms.RandomResizedCrop(224),
                                      transforms.RandomHorizontalFlip(0.5),
                                      transforms.ToTensor(),
                                      normalize])

In [ ]:
test_transform = transforms.Compose([transforms.Resize(224),
                                     transforms.ToTensor(),
                                     normalize])

In [ ]:
from torchvision.datasets import ImageFolder

In [ ]:
train_data = ImageFolder(root="../input/104-flowers-garden-of-eden/jpeg-224x224/train", transform=train_transform)

z,y = train_data[0]
print(z.shape,y)

plt.imshow(z.permute(1,2,0))

In [ ]:
train_dataloader = DataLoader(dataset=train_data, shuffle=True, batch_size=64)

In [ ]:
test_data = ImageFolder(root="../input/104-flowers-garden-of-eden/jpeg-224x224/val", transform=test_transform)

z,y = test_data[0]
print(z.shape, y)

plt.imshow(z.permute(1,2,0))

In [ ]:
test_dataloader = DataLoader(dataset=test_data,shuffle=True, batch_size=64)

In [ ]:
num_epoch = 5
batch_size= 64
learning_rate = 0.001

I dont want to wastetime by training from scratch so lets use pretrained model.

In [ ]:
from torchvision.models import googlenet

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
model = googlenet(pretrained=True)

In [ ]:
model.fc = nn.Linear(in_features=1024, out_features=104)

In [ ]:
model = model.to(device=device)

In [ ]:
loss_criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
for epoch in range(num_epoch):
    for data, target in train_dataloader:
        data = data.to(device=device)
        target = target.to(device=device)
        
        score = model(data)
        optimizer.zero_grad()
        loss = loss_criterion(score,target)
        
        loss.backward()
        
        optimizer.step()
    print(f"for epoch {epoch}, loss: {loss}")
        
        

In [ ]:
def check_accuracy(model, loader):
    model.eval()
    
    correct_sample = 0
    total_sample = 0
    
    for x, y  in loader:
        x = x.to(device=device)
        y = y.to(device=device)
        
        score = model(x)
        
        _, predictions = score.max(1)
        
        correct_sample = (y==predictions).sum()
        total_sample = predictions.shape[0]
    
    model.train()
    print(f"Total accuracy : {float(correct_sample/total_sample)*100}")
    

In [ ]:
check_accuracy(model, train_dataloader)

In [ ]:
check_accuracy(model, test_dataloader)